In [2]:
import requests
from bs4 import BeautifulSoup
import math
import time
from config import get_chrome_driver
from utils import fillForm, ProductSerializer
import pandas as pd

In [3]:
driver = get_chrome_driver()

In [4]:
def jumiaLogin(driver,email_address, user_password):
    url = 'https://www.jumia.co.ke/customer/account/login/'
    driver.get(url)
    time.sleep(5)
    create_acount =  driver.find_element_by_css_selector('a.btn._prim._i.-mtxl')
    time.sleep(2)
    create_acount.click()
    time.sleep(3)
    driver.back()
    time.sleep(10)
    email = driver.find_element_by_name("email")
    password = driver.find_element_by_name("password")
    login_btn = driver.find_element_by_css_selector('button.btn._prim._i.-mts')
    email.clear()
    time.sleep(2)
    password.clear()
    fillForm(email, email_address)
    time.sleep(6)
    fillForm(password,user_password)
    time.sleep(6)
    login_btn.click()

In [38]:

def get_resource(url, parameters, timeout=None):
    if timeout:
        html_string = requests.get(url,params=parameters,timeout=timeout)
    else:
        html_string = requests.get(url, params=parameters)
    return html_string
class JumiaScrapper():
    def __init__(self, search_timeout=None):
        #search_parameters => dict object {q:str,page:int,*kwargs}
        #jumia search url 
        self.url = 'https://www.jumia.co.ke/catalog/'
        self.search_timeout=search_timeout
        self.items_found = 0
        self.pages_found = 0
        self.dataframe = pd.DataFrame(columns=["product_name", "item_link","image_src","price","prev_price", "item_review", "rating", 'per_discount'])
        
    def call(self,filter_parameters,page_number='page_1'):
        html_string =get_resource(self.url, parameters=filter_parameters, timeout=self.search_timeout)
        soup = BeautifulSoup(html_string.text, 'lxml')
        title = soup.title.text.split('-')[0]
        articles= soup.find_all('article')  
        print(html_string.url)
        for i, article in enumerate(articles):
            item_link,prev_price, product_name, review, image_path,rating,price, per_discount =None, None, None,0, None, '0 out of 5', None, None
            try:
                price=article.find(class_='prc').text
            except:
                price=''
            if price != '':
                    item_link = ('https://www.jumia.co.ke{}').format(article.a['href'])
                    product_name = article.find(class_='name').text
                    try:
                        image_path=article.find('img')['data-src']
                        prev_price =  article.find(class_='old').text
                        per_discount = article.find(class_='tag _dsct _sm').text
                        review_info = article.find(class_='rev').text
                        rating = review_info.split('(')[0]
                        review = review_info.split('(')[-1].split(')')[0]
                    except:
                        pass
                    if prev_price == None:
                        prev_price=price
                        per_discount='0%'
                    item_data = ProductSerializer(name=product_name,item_link=item_link, 
                                                  image_path=image_path, price=price, 
                                                  prev_price=prev_price, item_reviews=review,
                                                  rating=rating, per_discount=per_discount)
                    page_dataframe = pd.DataFrame(item_data,index=[0])        
                    self.dataframe = pd.concat([self.dataframe,page_dataframe], ignore_index=True)
        if page_number=='page_1':
            items_found = soup.body.find(class_="-fs14 -gy5 -phs").text
            self.items_found = int(items_found.split(' ',1)[0])
            print(self.items_found, 'items found')
            self.pages_found =  math.ceil(self.items_found/len(articles)) 
            if self.url != html_string.url.split('?')[0]:
                self.url = html_string.url
                del filter_parameters['q']
        return title,  self.pages_found
    def get_result(self, filter_parameters, max_number_pages=5):
        title, num_pages = self.call(filter_parameters)
        for i in range(2,max_number_pages+1):
            if i<=num_pages:
                filter_parameters['page']=i
                _, _ = self.call(filter_parameters,page_number='page_{}'.format(i))       
        return {'title':title,'items_found':self.items_found,'pages_loaded':max_number_pages,
                'dataframe':self.dataframe}          

In [39]:
jumiascrapper = JumiaScrapper()
jumia_result = jumiascrapper.get_result(filter_parameters={'q':'tv 24 inch'},max_number_pages=4)

https://www.jumia.co.ke/televisions/
3315 items found
https://www.jumia.co.ke/televisions/
https://www.jumia.co.ke/televisions/?page=2
https://www.jumia.co.ke/televisions/?page=3
https://www.jumia.co.ke/televisions/?page=4
                                        product_name  \
0     EEFA 32LN4100D - 32" HD LED Digital TV - Black   
1        Haier 6005  - 24" HD LED Digital TV - Black   
2  EEFA 43LN4100S - 43'' Smart Android Digital LE...   
3     Hifinit By Haier 24" LED HD Digital TV - Black   
4                    UKA 32" HD Smart LED TV - Black   

                                           item_link  \
0  https://www.jumia.co.ke/eefa-32ln4100d-32-hd-l...   
1  https://www.jumia.co.ke/haier-6005-24-hd-led-d...   
2  https://www.jumia.co.ke/43ln4100s-43-smart-and...   
3  https://www.jumia.co.ke/hifinit-by-haier-24-le...   
4  https://www.jumia.co.ke/uka-32-hd-smart-led-tv...   

                                           image_src       price  prev_price  \
0  https://ke.jumia.is/

In [46]:
jumia_result['dataframe'].to_csv('jumia.csv', index=False)

In [47]:
pd.read_csv('jumia.csv').head()


,product_name,item_link,image_src,price,prev_price,item_review,rating,per_discount
0,"EEFA 32LN4100D - 32"" HD LED Digital TV - Black",https://www.jumia.co.ke/eefa-32ln4100d-32-hd-l...,https://ke.jumia.is/unsafe/fit-in/300x300/filt...,"KSh 10,999","KSh 12,999",189,4.4 out of 5,15%
1,"Haier 6005 - 24"" HD LED Digital TV - Black",https://www.jumia.co.ke/haier-6005-24-hd-led-d...,https://ke.jumia.is/unsafe/fit-in/300x300/filt...,"KSh 8,495","KSh 10,500",0,0 out of 5,19%
2,EEFA 43LN4100S - 43'' Smart Android Digital LE...,https://www.jumia.co.ke/43ln4100s-43-smart-and...,https://ke.jumia.is/unsafe/fit-in/300x300/filt...,"KSh 23,999","KSh 24,199",22,4.6 out of 5,1%
3,"Hifinit By Haier 24"" LED HD Digital TV - Black",https://www.jumia.co.ke/hifinit-by-haier-24-le...,https://ke.jumia.is/unsafe/fit-in/300x300/filt...,"KSh 7,399","KSh 13,999",1151,4.1 out of 5,47%
4,"UKA 32"" HD Smart LED TV - Black",https://www.jumia.co.ke/uka-32-hd-smart-led-tv...,https://ke.jumia.is/unsafe/fit-in/300x300/filt...,"KSh 13,999","KSh 20,000",1008,4 out of 5,30%


In [16]:
df

,age,name,year,id
0,24,ojay,1999,NaN
1,24,ojay,1999,NaN
2,24,ojay,1919,NaN
3,24,ojay,1919,1.0


In [25]:
df3=pd.DataFrame(data={"product_name":'item3', "item_link":'item_link',"image_src":None,
                                      "price":877,"prev_price":776, "item_review":8,
                                      "rating":76, 'per_discount':98},index=[0])

In [21]:
df2

,product_name,item_link,image_src,price,prev_price,item_review,rating,per_discount
0,nnhj,item_link,None,877,776,8,76,98


In [26]:
pd.concat([df,df2,df3], ignore_index=True)

,product_name,item_link,image_src,price,prev_price,item_review,rating,per_discount
0,nnhj,item_link,None,877,776,8,76,98
1,nnhj,item_link,None,877,776,8,76,98
2,item3,item_link,None,877,776,8,76,98
